In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle
import numpy as np

from art.attacks.evasion import FastGradientMethod, AutoAttack, AutoProjectedGradientDescent
from art.estimators.classification import KerasClassifier, TensorFlowV2Classifier
from art.utils import load_mnist
import foolbox as fb
import eagerpy as ep

/home/isaac/anaconda3/envs/tf/lib/python3.9/site-packages/art/estimators/certification/__init__.py:12: UserWarning: PyTorch not found. Not importing DeepZ functionality
  warnings.warn("PyTorch not found. Not importing DeepZ functionality")


In [2]:
test_path = '../Attention-based-Skin-Cancer-Classification/Ham10000 models/Augmentation/HAM10000_images_test'
batch_size= 16
datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input)
image_size = 224

print("\nTest Batches: ")
test_batches = datagen.flow_from_directory(test_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)


Test Batches: 
Found 828 images belonging to 7 classes.


In [4]:
for el in test_batches:
    print(np.max(el[0]))

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


KeyboardInterrupt: 

In [3]:
with open("../Attention-based-Skin-Cancer-Classification/Ham10000 models/Model without Soft Attention/Derm_model.pkl","rb") as file:
    model_without_attention = pickle.load(file)
with open("../Attention-based-Skin-Cancer-Classification/Ham10000 models/Model With Soft Attention/Derm+SA_model.pkl","rb") as file:
    model_with_attention = pickle.load(file)

2022-06-23 12:13:13.502107: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 12:13:13.529074: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 12:13:13.529238: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 12:13:13.530047: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [9]:
model = model_with_attention
preprocessing = dict()
bounds = (-1, 1)
fmodel = fb.TensorFlowModel(model, bounds=bounds, preprocessing=preprocessing)

2022-06-23 12:20:07.372924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 12:20:07.373259: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 12:20:07.373443: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 12:20:07.373680: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 12:20:07.373865: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [10]:
batch_accs = []
batch_weights = []
for i,(x,y) in zip(range(len(test_batches)),test_batches):
    # if i == 0:
    #     print(type(x),y)
    acc = fb.utils.accuracy(fmodel, x, tf.argmax(y, axis=1))
    batch_accs.append(acc)
    batch_weights.append(y.shape[0])
s1 = 0
s2 = sum(batch_weights)
for i in range(len(batch_accs)):
    s1 += batch_accs[i] * batch_weights[i]
print("unperturbed acc:",s1/s2)


unperturbed acc: 0.8804347823207505


In [11]:
attack = fb.attacks.LinfPGD()
adv_batches = []
labels = []
for i,(x,y) in zip(range(len(test_batches)),test_batches):
    raw, clipped, is_adv = attack(fmodel, tf.constant(x), tf.argmax(y, axis=1), epsilons=[0.00125])
    adv_batches.append(clipped[0])
    labels.append(tf.argmax(y, axis=1))

batch_accs = []
batch_weights = []
for (batch,label) in zip(adv_batches, labels):
    x = np.array(batch)
    y = np.array(label)
    acc = fb.utils.accuracy(fmodel, x, y)
    batch_accs.append(acc)
    batch_weights.append(y.shape[0])
s1 = 0
s2 = sum(batch_weights)
for i in range(len(batch_accs)):
    s1 += batch_accs[i] * batch_weights[i]
print("perturbed acc:",s1/s2)


perturbed acc: 0.7946859903381642


In [1]:
0.0025/2

0.00125

Using `LinfPGD()`
| Model/Epsilon     | 0                   | 0.00125             | 0.0025              | 0.005               | 0.01                |
| ----------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- |
| Without Attention | 0.9045893722686215  | 0.6871980679207954  | 0.21618357473525449 | 0.01207729472198348 | 0.0                 |
| With Attention    | 0.8804347823207505  | 0.7946859903381642  | 0.6847826089835973  | 0.336956521883103   | 0.021739130434782608|